# AdminGPT: Your AI-Powered Administrative Assistant, powered by OpenAI's Assistant Framework  🚀
### Introduction
AdmiGPT is an AI-powered administrative assistant, harnessing the power of OpenAI's Assistant framework to seamlessly integrate with your email and calendar. Similar to Microsoft's Copilot, only better, it's designed to be your ultimate productivity partner, AdmiGPT offers an array of advanced features, making your administrative tasks simpler, faster, and more efficient.

AdminGPT is fully Open Source, so everything you need to run it for yourself is in this Github repo. This notebook helps you get started with AdminGPT, and walks you through how it's implemented. 

### Implementation
To begin, we're going to load our custom OpenAI Tools, which will interface with your email platform's API, and store any confidential and authentication information for the user in environmental variables.

In [1]:
from openai import OpenAI
import time, json, pprint, os
from tools.o365_toolkit import (
    o365search_emails,
    o365search_email,
    o365search_events,
    o365parse_proposed_times,
    o365send_message,
    o365reply_message,
    o365send_event,
    o365find_free_time_slots,
    tools,
)
from datetime import datetime as dt

# Set your name
os.environ["CLIENT_NAME"] = "YOUR NAME"
# Set your email
os.environ["CLIENT_EMAIL"] = "YOUR EMAIL"
# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "YOUR KEY"
# Set your Microsoft Graph client ID
os.environ["CLIENT_ID"] = "YOUR ID"
# Set your Microsoft Graph client secret
os.environ["CLIENT_SECRET"] = "YOUR SECRET"

Next, we're going to set a few constants, which we will use throughout the code.

In [2]:
LOOP_DELAY_SECONDS = 2
debug = False
assistant_name = "Monica A. Ingenio"
model = "gpt-4-1106-preview"
current_date = dt.now()
formatted_date = current_date.strftime("%A, %B %d, %Y")
client_name = os.environ.get("CLIENT_NAME")
client_email = os.environ.get("CLIENT_EMAIL")
openai_api_key = os.environ.get("OPENAI_API_KEY")
assistant_instructions = (
    "You are an AI Administrative Assistant called "
    + assistant_name
    + ", and I am your executive. My name is "
    + client_name
    + ". My email is: "
    + client_email
    + ", in the Eastern Time (ET). You have access to my email and calendar. Today is "
    + formatted_date
    + ". "
)

# Add the debug prompt if user runs with debug
if debug:
    debug_prompt = (
        "Keep a record of any feedback requests provided by me"
        " detailing the prompt and tools calls in case I want to retrieve"
        " them."
    )
else:
    debug_prompt = ""
assistant_instructions = assistant_instructions + debug_prompt

The next step is to create an OpenAI Assistant so we can interact with it, and a thread in which to run our prompts.

In [3]:
client = OpenAI(
    api_key=openai_api_key,
)

assistant = client.beta.assistants.create(
    name="AI Administrative Assistant",
    instructions=assistant_instructions,
    model=model,
    tools=tools,
)

thread = client.beta.threads.create()

To make execution easier in the future steps, we create a function to run prompts with only one call. Then, we are going to use the function to submit the initial coaching prompt so that the Assistant gets better at performing tasks from the beginning. (A big part of the secret sauce of AdminGPT is in the coaching data file called coaching_data.txt, which gives table-stakes knowledge to the model)

In [4]:
def run_prompt(prompt: str, thread_id, assistant_id):
    message = client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=prompt,
    )

    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id,
    )
    return run


with open("coaching_data.txt", "r") as file:
    prompt = file = file.read()

run = run_prompt(prompt, thread.id, assistant.id)

To further make execution easier in future steps, we Create a function that polls the OpenAI API for a response to the prompt and executes tool calls, so we can use it to retrieve responses in the future.

In [5]:
def poll_for_response(thread_id, run_id):

    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        status = run.status

        if status == "completed":
            response = client.beta.threads.messages.list(thread_id=thread_id)
            if response.data:
                return response.data[0].content[0].text.value
            break
        elif status == "requires_action":
            tools_outputs = []

            for tool_call in run.required_action.submit_tool_outputs.tool_calls:
                tool_call_id = tool_call.id
                function_name = tool_call.function.name
                function_arguments = tool_call.function.arguments
                function_arguments = json.loads(function_arguments)

                # Case statement to execute each toolkit function
                if function_name == "o365search_emails":
                    output = o365search_emails(**function_arguments)
                elif function_name == "o365search_email":
                    output = o365search_email(**function_arguments)
                elif function_name == "o365search_events":
                    output = o365search_events(**function_arguments)
                elif function_name == "o365parse_proposed_times":
                    output = o365parse_proposed_times(
                        **function_arguments, client=client, model=model
                    )
                elif function_name == "o365send_message":
                    output = o365send_message(**function_arguments)
                elif function_name == "o365send_event":
                    output = o365send_event(**function_arguments)
                elif function_name == "o365reply_message":
                    output = o365reply_message(**function_arguments)
                elif function_name == "o365find_free_time_slots":
                    output = o365find_free_time_slots(**function_arguments)

                # Clean the function output into JSON-like output
                output = pprint.pformat(output)
                tool_output = {"tool_call_id": tool_call_id, "output": output}
                tools_outputs.append(tool_output)

            if run.required_action.type == "submit_tool_outputs":
                client.beta.threads.runs.submit_tool_outputs(
                    thread_id=thread_id, run_id=run_id, tool_outputs=tools_outputs
                )

        elif status == "failed":
            return "Run failed try again!"
            break

        time.sleep(LOOP_DELAY_SECONDS)

Now that we've submit our first prompt, we are going to poll for a response. If the response is, "How can I help you?", we know that the coaching data was sent correctly, and we are ready to start using AdminGPT.

In [6]:
response = poll_for_response(thread.id, run.id)
print(response)

How can I help you?


To begin, we're going to perform the most simple task, which is summarizing an email from a specific sender.

In [7]:
prompt = (
    "Can you please concisely summarize the most recent email from Santiago Delgado"
    " including any proposed meeting times?"
)
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

The most recent email from Santiago Delgado expresses curiosity about the possibility of Mike Portnoy rejoining Dream Theater. Santiago Delgado proposes a discussion on this topic and suggests meeting on February 3, 2024, at 4:00 pm ET. He recalls Mike Portnoy's significant role in the band's development and his departure in 2010 that led to his involvement in various other musical projects. Santiago Delgado is looking forward to hearing thoughts on whether Mike Portnoy's potential return could cause a resurgence of the band's classic sound or lead to unforeseen complications.


Now, we're going to show AdminGPT's ability to interact with your calendar. We're going to check what events we have on the day that the email proposed.

In [8]:
prompt = (
    "What meetings do I have on the day that Santiago Delgado is proposing to meet in"
    " his most email?"
)
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

/home/codespace/.python/current/lib/python3.10/site-packages/O365/utils/windows_tz.py:638: PytzUsageWarning: The zone attribute is specific to pytz's interface; please migrate to a new time zone provider. For more details on how to do so, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  iana_tz.zone if isinstance(iana_tz, tzinfo) else iana_tz)


On February 3, 2024, the day Santiago Delgado proposes to meet, you have the following meetings scheduled:

1. **Flight from Dallas to Cincinnati**
   - From: 8:00 am ET
   - Until: 10:30 am ET

2. **Lunch Meeting with Management**
   - From: 12:00 pm ET
   - Until: 1:00 pm ET

3. **Strategy Session**
   - From: 3:00 pm ET
   - Until: 5:00 pm ET

4. **Flight from Cincinnati to Dallas**
   - From: 6:00 pm ET
   - Until: 8:30 pm ET

Given this schedule, the proposed meeting time of 4:00 pm ET on February 3 intersects with the Strategy Session you have from 3:00 pm to 5:00 pm ET.


We now know when the email sender wants to meet, and what meetings I have on that day. So, let's see what times I have free that day.

In [9]:
prompt = "What times am I free on the day that Santiago is proposing to meet?"
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

On February 3, 2024, you are free during the following time slots:

1. From midnight to 8:00 am ET
2. From 10:30 am to 12:00 pm ET
3. From 1:00 pm to 3:00 pm ET
4. From 5:00 pm to 6:00 pm ET
5. From 8:30 pm to 11:59:59 pm ET

Therefore, considering the proposed meeting time of 4:00 pm ET by Santiago Delgado, you are not free as it clashes with your existing "Strategy Session" from 3:00 pm to 5:00 pm ET. However, you do have a free slot available just after that meeting, from 5:00 pm ET to 6:00 pm ET.


With all this information, we can now draft a response to the email letting the sender know that we can't meet at the proposed time, and propose other times to meet.

In [10]:
prompt = (
    "Please draft a response to Santiago letting him know that I can't meet on the"
    " proposed times, and propose other free times on the same day. Show me the draft."
)
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

The draft response to Santiago has been saved with the subject "Mike Portnoy coming back to Dream Theater?". Here is the content of the draft:

---

Hi Santiago,

Thank you for reaching out. Unfortunately, I have a prior engagement during the time you suggested for our discussion on Mike Portnoy's potential return to Dream Theater. However, I am available before and after certain appointments on the same day. Would any of the following times work for you?

- From 10:30 am to 12:00 pm ET
- From 1:00 pm to 3:00 pm ET
- From 5:00 pm to 6:00 pm ET
- After 8:30 pm ET

Please let me know your preference and I'll make sure to block out the time on my calendar.

Looking forward to our conversation.

Best regards,

Santiago Delgado

---

Let me know if you would like to send this draft or if you need any adjustments.


Sometimes, we will want the email to come from our AI administrative instead of the ourselves, so we can do that too.

In [11]:
prompt = "Can you edit the email and have it come from you on behalf of me?"
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

Certainly, I will revise the draft so that it comes from me on your behalf. Here is the updated content:

---

Hi Santiago,

I hope you are doing well. I am writing to you on behalf of Santiago Delgado to discuss your recent email regarding Mike Portnoy potentially rejoining Dream Theater. Unfortunately, Santiago is not available at the proposed time of 4:00 pm ET on February 3, 2024, due to prior commitments.

However, he would be happy to discuss this with you at alternative times on the same day. The following slots are open on his calendar:

- From 10:30 am to 12:00 pm ET
- From 1:00 pm to 3:00 pm ET
- From 5:00 pm to 6:00 pm ET
- After 8:30 pm ET

Please let me know if any of these times would be convenient for you, and we will schedule the meeting accordingly.

Thank you for your understanding, and we look forward to the possibility of this discussion.

Warm regards,

Monica A. Ingenio
Administrative Assistant to Santiago Delgado

---

Before proceeding, does this updated draft m

To conclude, I want to show that you can follow all these steps in one prompt.

In [12]:
prompt = (
    "Can you draft a response to Santiago Delgado's most recent email from you on"
    " behalf of me letting him know if I can meet at the times he proposes, and if not,"
    " propose other times on the same day? Show me the draft."
)
run = run_prompt(prompt, thread.id, assistant.id)
response = poll_for_response(thread.id, run.id)
print(response)

The draft response to Santiago Delgado has been saved with the subject "Mike Portnoy coming back to Dream Theater?". Here is the content of the draft:

---

Dear Santiago Delgado,

This is Monica A. Ingenio, Santiago Delgado's administrative assistant at ACON Investments. Unfortunately, Santiago has a prior commitment during the proposed time of 4:00 pm ET on February 3rd, 2024, and will not be able to meet then.

However, he is available at other times on the same day and would be pleased to discuss the topic of Mike Portnoy and Dream Theater with you. Please find below the times he is available:

- From 10:30 am to 12:00 pm ET
- From 1:00 pm to 3:00 pm ET
- From 5:00 pm to 6:00 pm ET
- After 8:30 pm ET

Please let us know which time works best for you so we can schedule the meeting accordingly.

Warm regards,

Monica A. Ingenio
Administrative Assistant to Santiago Delgado
ACON Investments

---

Let me know if this draft meets your expectations or if you would like any further changes